# Gravitational-Wave Sky Localizations: Interactive Sky Map Visualizations  and Comparisons.


The tutorial aims to show a practical approach for working with a Gravitational-Wave (GW) sky localization
when the enclosed area within a given probability level contour, the credible region, is encoded with the [Multi-Order Coverage](http://ivoa.net/documents/MOC/) (MOC) map. MOC is an
[IVOA](http://ivoa.net/) standard since 2015 and it is based on the [HEALPix](https://healpix.sourceforge.io/) tessellation of the sky.
 
We will perform the following steps.

1) Encode a credible regions of a GW sky localization with a MOC map.

2) Synchronize comparison between the BAYESTAR and LALinference sky maps. 

3) Intersection area between BAYESTAR and LALinference sky maps. 


## Requirements
The tutorial runs only in Python 3. For running it, install the following modules.


[ligo.skymap](https://lscsoft.docs.ligo.org/ligo.skymap/) ≥ 0.4.0

[mocpy](https://github.com/cds-astro/mocpy) ≥ 0.8.4

[ipyaladin](https://github.com/cds-astro/ipyaladin)

[astropy](https://www.astropy.org/) ≥ 3.1

## 1. Encode a credible region contours of a GW sky localization with a MOC map
We download the sky map from the [GraceDB](https://gracedb.ligo.org/) (Gravitational-Wave Candidate Event Database) under the section [Public Alert](https://gracedb.ligo.org/superevents/public/O3/). The Superevent  [S190814bv](https://gracedb.ligo.org/superevents/S190814bv/view/), confirmed as [GW190814](https://iopscience.iop.org/article/10.3847/2041-8213/ab960f), is chosen here. We select the [BAYESTAR sky map](https://gracedb.ligo.org/apiweb/superevents/S190814bv/files/bayestar.multiorder.fits,1) provided by the 3 interferometers: Virgo, LIGO Hanford and LIGO Livingston.
The credible regions at the probability level contours of 90%, 50% and 10% will be created and visualized in an Aladin widget.

We will use the [ligo-skymap-contour-moc](https://lscsoft.docs.ligo.org/ligo.skymap/tool/ligo_skymap_contour_moc.html) method implemented in [ligo.skymap](https://lscsoft.docs.ligo.org/ligo.skymap/). The method creates a contour for a credible level of an all-sky probability map. The input is a HEALPix FITS ([flatten or multi-resolution](https://emfollow.docs.ligo.org/userguide/tutorial/multiorder_skymaps.html)) probability map. The output is a Multi-Order Coverage (MOC) serialized in a FITS file.

In [1]:
# Download the sky map from GraceDB
from astropy.utils.data import download_file

gracedb_url_gw190814_bayestar_3ifo = ('https://gracedb.ligo.org/api/superevents/'
                                      'S190814bv/files/bayestar.multiorder.fits')
gw190814_bayestar_3ifo = download_file(gracedb_url_gw190814_bayestar_3ifo)

In [2]:
# import the method "ligo_skymap_contour_moc" from "ligo.skymap"
from ligo.skymap.tool.ligo_skymap_contour_moc import main

# Define 3 credible regions at the probability level contours of 90% - 50% - 10%.
contours = ['90', '50', '10']

# Name the output files of these probability level contours.
output_files = ['GW190814-BAYESTAR (90%)', 'GW190814-BAYESTAR (50%)', 'GW190814-BAYESTAR (10%)']

# Create and save the credible regions files (.fits) encoded with MOC
for contour, output_file in zip(contours, output_files):
    main([gw190814_bayestar_3ifo, '--contour', contour, '--output', output_file])

### 1.1 Interactive Sky Map Visualization: Aladin Widget

An Aladin  widget is initialized at a defined target position with a DSS color survey background. Additional background images can be selected by the **Manage Layers**.

In [3]:
# Initialize an Aladin widget at a defined sky's position and with the DSS color survey.
import ipyaladin as ipyal

aladin = ipyal.Aladin(target='16.91000 -28.1100', fov=35, survey='P/DSS2/color')
aladin

Aladin(fov=35.0, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'o…

In [4]:
# Show the credible regions in the Aladin widget with 3 different colors.
colors = ["blue", "yellow", "red"]

for output_file, color in zip(output_files, colors):
    aladin.add_moc_from_URL(output_file, {'color': color, 'opacity': 0.7,
                            'adaptativeDisplay': False, 'name':output_file})

Scroll up and visualize the credible region contours in the interactive Aladin widget. From the **Manage Layers**, the 3 credible regions can be independently selected. 
If you hover the mouse pointer over the credible region plan a message will be displayed above it to show the coverage in sky percentage.

## 2. Synchronized comparison between the BAYESTAR and LALinference sky maps.

We synchronize 3 widgets to compare the 90% credible region contours between the BAYESTAR, LALinference
and the final sky localization presented in LIGO and Virgo paper [GW190814: Gravitational Waves from the Coalescence of a 23 Solar Mass Black Hole with a 2.6 Solar Mass Compact Object](https://iopscience.iop.org/article/10.3847/2041-8213/ab960f).

In [5]:
from ipyaladin import Aladin
from ipywidgets import Layout, Box, widgets

# Initialize the boxes
left_box = Aladin(layout=Layout(width='33%'), target='17.3800 -29.1100', fov=30,survey='P/DSS2/color')
middle_box = Aladin(layout=Layout(width='33%'))
right_box = Aladin(layout=Layout(width='33%'))

# Synchronize target between 3 widgets
widgets.jslink((left_box, 'target'), (middle_box, 'target'))
widgets.jslink((middle_box, 'target'), (right_box, 'target'))

# Synchronize FoV (zoom level) between 2 widgets
widgets.jslink((left_box, 'fov'), (middle_box, 'fov'))
widgets.jslink((middle_box, 'fov'), (right_box, 'fov'))

items = [left_box, middle_box, right_box]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box = Box(children=items, layout=box_layout)
box

Box(children=(Aladin(fov=30.0, layout=Layout(width='33%'), options=['allow_full_zoomout', 'coo_frame', 'fov', …

The 90% credible region contour of the BAYESTAR sky map is generated in the cells above. Here we download the LALinference and the last sky map from GraceDB and finally we generated the corrisponding 90% credible region for both.

In [6]:
# Download the LALinference sky map from the GraceDB URL
gracedb_url_gw190814_lalinference = ('https://gracedb.ligo.org/api/superevents/'
                                     'S190814bv/files/LALInference.v1.multiorder.fits,0')
gw190814_lalinference = download_file(gracedb_url_gw190814_lalinference)

# Create and save the credible regions .fits file encoded with MOC
main([gw190814_lalinference, '--contour', '90', '--output', 'GW190814-LALinference (90%)'])

In [7]:
# Download the final sky map presented in the LVC paper from the GraceDB URL 
gracedb_url_gw190814_phm = ('https://gracedb.ligo.org/api/superevents/'
                            'S190814bv/files/GW190814_skymap.fits.gz,0')
gw190814_phm = download_file(gracedb_url_gw190814_phm)

# Create and save the credible regions .fits file encoded with MOC
main([gw190814_phm, '--contour', '90', '--output', 'GW190814-PHM (90%)'])

In [8]:
# Show the bayestar sky map (90% credible region)
left_box.add_moc_from_URL('GW190814-BAYESTAR (90%)', {'color': 'blue', 'opacity': 0.8, 
                                      'adaptativeDisplay': False, 'name': 'GW190814-BAYESTAR (90%)' })

# Show the lalinference sky map (90% credible region)
middle_box.add_moc_from_URL('GW190814-LALinference (90%)', {'color': 'orange', 'opacity': 0.8, 
                                      'adaptativeDisplay': False, 'name': 'GW190814-LALinference (90%)' })

# Show lalinference and bayestar overlapped
right_box.add_moc_from_URL('GW190814-PHM (90%)', {'color': 'magenta', 'opacity': 0.8, 
                                      'adaptativeDisplay': False, 'name': 'GW190814-PHM (90%)' })

Now we measure the corrisponding areas in square degrees.

In [9]:
# Import mocpy and check the version
import mocpy
from mocpy import MOC
mocpy.__version__

'0.8.4'

In [10]:
# square degrees in a whole sphere
from math import pi
square_degrees_sphere = (360.0**2)/pi

# Load and measure BAYESTAR sky coverage 90 c.r.
gw190814_bayestar_90 = MOC.from_fits("GW190814-BAYESTAR (90%)")

sky_fraction_bayestar = gw190814_bayestar_90.sky_fraction
print("The 90% area covered by the BAYESTAR 90% credible region is", 
      round(sky_fraction_bayestar * square_degrees_sphere, 1), 'deg^2')

# Load and measure LALinference sky coverage 90% c.r.
gw190814_lalinference_90 = MOC.from_fits("GW190814-LALinference (90%)")

sky_fraction_lalinference = gw190814_lalinference_90.sky_fraction
print("The 90% area covered by the LALinference 90% credible region is", 
      round(sky_fraction_lalinference * square_degrees_sphere, 1), 'deg^2')

# Load and measure the final sky coverage 90% c.r.
gw190814_phm_90 = MOC.from_fits("GW190814-PHM (90%)")

sky_fraction_phm = gw190814_phm_90.sky_fraction
print("The 90% area covered by the final sky map (PHM) 90% credible region is", 
      round(sky_fraction_phm * square_degrees_sphere, 1),'deg^2')

The 90% area covered by the BAYESTAR 90% credible region is 37.6 deg^2
The 90% area covered by the LALinference 90% credible region is 23.1 deg^2
The 90% area covered by the final sky map (PHM) 90% credible region is 18.5 deg^2


## 2.1 Overlap Visualization

We display the 3 sky maps in a single Aladin widget. From the **Manage Layers**, the credible regions of the 3 sky localizations (90% probability area) can be independently selected. 

In [11]:
# Initialize the Aladin widget at a defined sky's position and with the DSS color survey.
import ipyaladin as ipyal

aladin = ipyal.Aladin(target='16.91000 -28.1100', fov=50, survey='P/DSS2/color')
aladin

Aladin(fov=50.0, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'o…

In [12]:
# Show the credible regions for the 3 sky maps in the Aladin widget.
colors = ["blue", "orange", "magenta"]

credible_regions = ['GW190814-BAYESTAR (90%)', 'GW190814-LALinference (90%)', 
                    'GW190814-PHM (90%)']

for credible_region, color in zip(credible_regions, colors):
    aladin.add_moc_from_URL(credible_region, {'color': color, 'opacity': 0.9,
                            'adaptativeDisplay': False, 'name':credible_region})

## 3 Intersection Area

Finally we measure the intersection area between the BAYESTAR and LALinference sky maps at the 90% credible regions.
A new MOC file (.fits) is created named *bayestar_lalinference_intersection* and it is displayed in the last initialated Aladin widget.

In [13]:
# Intersection between BAYESTAR and LALinference sky maps (90% credible regions)
bayestar_lalinference_intersection = gw190814_bayestar_90.intersection(gw190814_lalinference_90)

# Write the MOC .fits file
bayestar_lalinference_intersection.write("bayestar_lalinference_intersection", 
                                         overwrite=True, format='fits')

In [14]:
# Load, measure and print the intersection between BAYESTAR and LALinference sky maps
bayestar_lalinference_intersection = MOC.from_fits("bayestar_lalinference_intersection")

sky_fraction_int = bayestar_lalinference_intersection.sky_fraction

print("The intersection area between the BAYESTAR and the LALinference sky maps is", 
      round(sky_fraction_int * square_degrees_sphere, 1), 'deg^2')

The intersection area between the BAYESTAR and the LALinference sky maps is 23.1 deg^2


In [15]:
# Visualize the intersection region in the last Aladin widget
aladin.add_moc_from_URL('bayestar_lalinference_intersection', {'color': 'yellow', 'opacity': 0.9,
                                                               'adaptativeDisplay': False, 
                                                               'name':'bayestar_lalinference_intersection'})

That intersection region can be indipendently selected from the **Manage Layers**.

